In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_extract, when
from pyspark.sql.types import IntegerType



In [ ]:
# Define file locations
pin_file_location = "dbfs:/mnt/mount_name/topics/topics/1209b9ad90a5.pin/partition=0/*.json"
geo_file_location = "dbfs:/mnt/mount_name/topics/topics/1209b9ad90a5.geo/partition=0/*.json"
user_file_location = "dbfs:/mnt/mount_name/topics/topics/1209b9ad90a5.user/partition=0/*.json"

# Read JSON files into DataFrames
df_pin = spark.read.json(pin_file_location)
df_geo = spark.read.json(geo_file_location)
df_user = spark.read.json(user_file_location)

## Task 1: Clean the DataFrame that contains information about Pinterest posts

#### To clean the df_pin DataFrame you should perform the following transformations:

- Replace empty entries and entries with no relevant data in each column with Nones
- Perform the necessary transformations on the follower_count to ensure every entry is a number. Make sure the data type of this column is an int.
- Ensure that each column containing numeric data has a numeric data type
- Clean the data in the save_location column to include only the save location path
- Rename the index column to ind.
- Reorder the DataFrame columns to have the following column order:
  - ind
  - unique_id
  - title
  - description
  - follower_count
  - poster_name
  - tag_list
  - is_image_or_video
  - image_src
  - save_location
  - category

In [ ]:
# Show the DataFrames
df_pin.show()
df_geo.show()
df_user.show()

+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 category| description|downloaded|follower_count| image_src|index|is_image_or_video| poster_name| save_location| tag_list| title| unique_id|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 travel|This Costa Rica i...| 1| 10k|https://i.pinimg....|10138| image|Wanderlust Chloe ...|Local save in /da...|Costa Rica Travel...|14 Amazing Things...|927c4658-cc3f-4b9...|
diy-and-crafts|This post may con...| 1| 892k|https://i.pinimg....| 3156| image|Michelle {CraftyM...|Local save in /da...|Christmas Gifts F...|Handprint Reindee...|fa6e31a4-18c2-4ec...|
 finance|Der #Vermögensauf...| 1| 3k|https://i.pinimg....| 5395| image|DIVDepot – Geld s...|Local save in /da...|Money Saving Chal...|So startest du mi...|71f72304-3708-45f...|
 finance|If you love budge...| 1| 26k|https://i.pinimg....| 5494| image|Living Low Key | ...|Local save in /da...|Financial Peace,F...|Dave Ramsey's 7 B...|8fb2af68-543b-463...|
 christmas|Here are the best...| 1| 500k|https://i.pinimg....| 2418| image| HikenDip|Local save in /da...|Farmhouse Christm...|100 DIY Christmas...|da8745a6-5160-46c...|
 beauty|[PaidLink] It's m...| 1| 22|https://i.pinimg....| 1611| image| Hair Accessories|Local save in /da...|Natural Beauty Ti...|Over The Years, P...|051e231c-7509-437...|
 education|Tired of Google C...| 1| 9k|https://i.pinimg....| 3813| image|Science of Curiosity|Local save in /da...|Google Classroom,...|Fun Interactive D...|49f7dfe7-2e5a-403...|
 quotes|summcoco gives yo...| 1| 306k|https://i.pinimg....| 7922| image|Sumcoco | Decor I...|Local save in /da...|Life Quotes Love,...|45 Top Life Quote...|a584581c-1b38-473...|
 christmas|Connecticut life ...| 1| 84k|https://i.pinimg....| 1643| image| Lauren McBride|Local save in /da...|Christmas Dining ...|White and Gold Ch...|41be1bcd-1ead-476...|
 finance|Buying your first...| 1| 223|https://i.pinimg....| 5091| image|Edwin | Cash The ...|Local save in /da...|Paying Off Car Lo...|Buying Your First...|43459535-64e9-48a...|
 travel|Are you traveling...| 1| 3k|https://i.pinimg....| 9979| image| Petite in Paris|Local save in /da...|Torre Eiffel Pari...|Paris in the Summ...|2b2abc85-fc51-481...|
diy-and-crafts|This ghost craft ...| 1| 56k|https://i.pinimg....| 3032| video| Thriving Home|Local save in /da...|Halloween Arts An...| Puffy Ghost Craft|208b07bc-e042-4e8...|
 mens-fashion|It's safe to assu...| 1| 383k|https://i.pinimg....| 6809| image| Suburban Men|Local save in /da...|Business Casual D...|It's Time to Ditc...|f8ca698f-abfc-4b6...|
 education|Amazon is a teach...| 1| 41k|https://i.pinimg....| 3644| image|Teach Create Moti...|Local save in /da...|Classroom Hacks,5...|More Amazon Must ...|cde2b4e2-7711-4a8...|
 home-decor|Believe it or not...| 1| 31k|https://i.pinimg....| 6575| image| Maria Louise Design|Local save in /da...|Farmhouse Remodel...|Affordable Farmho...|15c45907-657e-439...|
 home-decor|Holiday mantle de...| 1| 83k|https://i.pinimg....| 6145| image| Stylin by Aylin|Local save in /da...|Winter Home Decor...|HOLIDAY MANTLE DE...|82e13a07-db99-43a...|
 home-decor|Holiday mantle de...| 1| 83k|https://i.pinimg....| 6145| image| Stylin by Aylin|Local save in /da...|Winter Home Decor...|HOLIDAY MANTLE DE...|82e13a07-db99-43a...|
 art|Mexican artist En...| 1| 2M|https://i.pinimg....| 4| image| Bored Panda|Local save in /da...|Girl Drawing Sket...|Mexican Artist Us...|55abcd28-bda1-445...|
 education|Podcasts for Teac...| 1| 25k|https://i.pinimg....| 4315| image| Math Giraffe|Local save in /da...|Middle School Cla...|Podcasts for Teac...|21b59ba9-829d-4c3...|
event-planning|Sacramento Califo...| 1| 6M|https://i.pinimg....| 5069| image| Style Me

In [ ]:
df_pin.describe().show()
df_geo.describe().show()
df_user.describe().show()

+-------+--------+--------------------+-------------------+-----------------+--------------------+-----------------+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+
summary|category| description| downloaded| follower_count| image_src| index|is_image_or_video| poster_name| save_location| tag_list| title| unique_id|
+-------+--------+--------------------+-------------------+-----------------+--------------------+-----------------+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+
 count| 65| 65| 65| 65| 65| 65| 65| 65| 65| 65| 65| 65|
 mean| null| null| 0.9538461538461539|387.7142857142857| null|5666.861538461539| null| null| null| null| null| null|
 stddev| null| null|0.21145103088313016|393.8793022766045| null|3363.418963072226| null| null| null| null| null| null|
 min| art|#lovequotes #matc...| 0| 0| Image src error.| 4| image|ALL FOR WOMAN|Local save in /da...|60th Anniversary ...|100 DIY Christmas...|051e231c-7509-437...|
 max|vehicles|summcoco gives yo...| 1| User Info Error|https://i.pinimg....| 10883| video| wikiHow|Local save in /da...|Wise Quotes,Quota...|mix and match - H...|fbe53c66-3442-477...|
+-------+--------+--------------------+-------------------+-----------------+--------------------+-----------------+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+

+-------+-----------+-----------------+------------------+-------------------+-------------------+
summary| country| ind| latitude| longitude| timestamp|
+-------+-----------+-----------------+------------------+-------------------+-------------------+
 count| 65| 65| 65| 65| 65|
 mean| null|5666.861538461539|-47.95451683076923|-110.52550615384615| null|
 stddev| null|3363.418963072226| 49.96288705461774| 73.4911784661402| null|
 min|Afghanistan| 4| -89.9787| -179.779|2018-01-13 19:33:49|
 max| Uganda| 10883| 88.2631| 96.2413|2022-10-09 08:06:50|
+-------+-----------+-----------------+------------------+-------------------+-------------------+

+-------+-----------------+-------------------+----------+-----------------+---------+
summary| age| date_joined|first_name| ind|last_name|
+-------+-----------------+-------------------+----------+-----------------+---------+
 count| 100| 100| 100| 100| 100|
 mean| 28.14| null| null| 5810.16| null|
 stddev|7.877881066696177| null| null|3168.080506450797| null|
 min| 20|2015-10-21 08:27:36| Aaron| 4| Abbott|
 max| 53|2017-10-10 20:09:33| Thomas| 10883| Waller|
+-------+-----------------+-------------------+----------+-----------------+---------+

In [ ]:
print(df_pin.columns)
print(df_geo.columns)
print(df_user.columns)

['category', 'description', 'downloaded', 'follower_count', 'image_src', 'index', 'is_image_or_video', 'poster_name', 'save_location', 'tag_list', 'title', 'unique_id']
['country', 'ind', 'latitude', 'longitude', 'timestamp']
['age', 'date_joined', 'first_name', 'ind', 'last_name']

In [ ]:
#Check data types
print(df_pin.dtypes)
print(df_geo.dtypes)
print(df_user.dtypes)

[('category', 'string'), ('description', 'string'), ('downloaded', 'bigint'), ('follower_count', 'string'), ('image_src', 'string'), ('index', 'bigint'), ('is_image_or_video', 'string'), ('poster_name', 'string'), ('save_location', 'string'), ('tag_list', 'string'), ('title', 'string'), ('unique_id', 'string')]
[('country', 'string'), ('ind', 'bigint'), ('latitude', 'double'), ('longitude', 'double'), ('timestamp', 'string')]
[('age', 'bigint'), ('date_joined', 'string'), ('first_name', 'string'), ('ind', 'bigint'), ('last_name', 'string')]

In [ ]:
# Replace empty entries and entries with no relevant data in each column with None for df_pin DataFrame
from pyspark.sql.functions import col
df_pin = df_pin.fillna("None")

In [ ]:
df_pin.show()

+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 category| description|downloaded|follower_count| image_src|index|is_image_or_video| poster_name| save_location| tag_list| title| unique_id|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 travel|This Costa Rica i...| 1| 10k|https://i.pinimg....|10138| image|Wanderlust Chloe ...|Local save in /da...|Costa Rica Travel...|14 Amazing Things...|927c4658-cc3f-4b9...|
diy-and-crafts|This post may con...| 1| 892k|https://i.pinimg....| 3156| image|Michelle {CraftyM...|Local save in /da...|Christmas Gifts F...|Handprint Reindee...|fa6e31a4-18c2-4ec...|
 finance|Der #Vermögensauf...| 1| 3k|https://i.pinimg....| 5395| image|DIVDepot – Geld s...|Local save in /da...|Money Saving Chal...|So startest du mi...|71f72304-3708-45f...|
 finance|If you love budge...| 1| 26k|https://i.pinimg....| 5494| image|Living Low Key | ...|Local save in /da...|Financial Peace,F...|Dave Ramsey's 7 B...|8fb2af68-543b-463...|
 christmas|Here are the best...| 1| 500k|https://i.pinimg....| 2418| image| HikenDip|Local save in /da...|Farmhouse Christm...|100 DIY Christmas...|da8745a6-5160-46c...|
 beauty|[PaidLink] It's m...| 1| 22|https://i.pinimg....| 1611| image| Hair Accessories|Local save in /da...|Natural Beauty Ti...|Over The Years, P...|051e231c-7509-437...|
 education|Tired of Google C...| 1| 9k|https://i.pinimg....| 3813| image|Science of Curiosity|Local save in /da...|Google Classroom,...|Fun Interactive D...|49f7dfe7-2e5a-403...|
 quotes|summcoco gives yo...| 1| 306k|https://i.pinimg....| 7922| image|Sumcoco | Decor I...|Local save in /da...|Life Quotes Love,...|45 Top Life Quote...|a584581c-1b38-473...|
 christmas|Connecticut life ...| 1| 84k|https://i.pinimg....| 1643| image| Lauren McBride|Local save in /da...|Christmas Dining ...|White and Gold Ch...|41be1bcd-1ead-476...|
 finance|Buying your first...| 1| 223|https://i.pinimg....| 5091| image|Edwin | Cash The ...|Local save in /da...|Paying Off Car Lo...|Buying Your First...|43459535-64e9-48a...|
 travel|Are you traveling...| 1| 3k|https://i.pinimg....| 9979| image| Petite in Paris|Local save in /da...|Torre Eiffel Pari...|Paris in the Summ...|2b2abc85-fc51-481...|
diy-and-crafts|This ghost craft ...| 1| 56k|https://i.pinimg....| 3032| video| Thriving Home|Local save in /da...|Halloween Arts An...| Puffy Ghost Craft|208b07bc-e042-4e8...|
 mens-fashion|It's safe to assu...| 1| 383k|https://i.pinimg....| 6809| image| Suburban Men|Local save in /da...|Business Casual D...|It's Time to Ditc...|f8ca698f-abfc-4b6...|
 education|Amazon is a teach...| 1| 41k|https://i.pinimg....| 3644| image|Teach Create Moti...|Local save in /da...|Classroom Hacks,5...|More Amazon Must ...|cde2b4e2-7711-4a8...|
 home-decor|Believe it or not...| 1| 31k|https://i.pinimg....| 6575| image| Maria Louise Design|Local save in /da...|Farmhouse Remodel...|Affordable Farmho...|15c45907-657e-439...|
 home-decor|Holiday mantle de...| 1| 83k|https://i.pinimg....| 6145| image| Stylin by Aylin|Local save in /da...|Winter Home Decor...|HOLIDAY MANTLE DE...|82e13a07-db99-43a...|
 home-decor|Holiday mantle de...| 1| 83k|https://i.pinimg....| 6145| image| Stylin by Aylin|Local save in /da...|Winter Home Decor...|HOLIDAY MANTLE DE...|82e13a07-db99-43a...|
 art|Mexican artist En...| 1| 2M|https://i.pinimg....| 4| image| Bored Panda|Local save in /da...|Girl Drawing Sket...|Mexican Artist Us...|55abcd28-bda1-445...|
 education|Podcasts for Teac...| 1| 25k|https://i.pinimg....| 4315| image| Math Giraffe|Local save in /da...|Middle School Cla...|Podcasts for Teac...|21b59ba9-829d-4c3...|
event-planning|Sacramento Califo...| 1| 6M|https://i.pinimg....| 5069| image| Style Me

In [ ]:
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import IntegerType

# Replace "k" with "000" and cast to integer
df_pin = df_pin.withColumn("follower_count", 
                           regexp_replace(col("follower_count"), "k", "000").cast(IntegerType())
                          )


In [ ]:
print(df_pin)

DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string]

In [ ]:
from pyspark.sql.types import IntegerType, LongType

# Cast downloaded column to IntegerType
df_pin = df_pin.withColumn("downloaded", df_pin["downloaded"].cast(IntegerType()))

# Cast index column to LongType
df_pin = df_pin.withColumn("index", df_pin["index"].cast(LongType()))


In [ ]:
# Rename index column to ind
df_pin = df_pin.withColumnRenamed("index", "ind")


In [ ]:
from pyspark.sql.types import IntegerType, LongType

# Cast downloaded column to IntegerType
df_pin = df_pin.withColumn("downloaded", df_pin["downloaded"].cast(IntegerType()))

# Cast index column to LongType
df_pin = df_pin.withColumn("index", df_pin["index"].cast(LongType()))


+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 category| description|downloaded|follower_count| image_src| ind|is_image_or_video| poster_name| save_location| tag_list| title| unique_id|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 travel|This Costa Rica i...| 1| 10000|https://i.pinimg....|10138| image|Wanderlust Chloe ...| /data/travel|Costa Rica Travel...|14 Amazing Things...|927c4658-cc3f-4b9...|
diy-and-crafts|This post may con...| 1| 892000|https://i.pinimg....| 3156| image|Michelle {CraftyM...|/data/diy-and-crafts|Christmas Gifts F...|Handprint Reindee...|fa6e31a4-18c2-4ec...|
 finance|Der #Vermögensauf...| 1| 3000|https://i.pinimg....| 5395| image|DIVDepot – Geld s...| /data/finance|Money Saving Chal...|So startest du mi...|71f72304-3708-45f...|
 finance|If you love budge...| 1| 26000|https://i.pinimg....| 5494| image|Living Low Key | ...| /data/finance|Financial Peace,F...|Dave Ramsey's 7 B...|8fb2af68-543b-463...|
 christmas|Here are the best...| 1| 500000|https://i.pinimg....| 2418| image| HikenDip| /data/christmas|Farmhouse Christm...|100 DIY Christmas...|da8745a6-5160-46c...|
 beauty|[PaidLink] It's m...| 1| 22|https://i.pinimg....| 1611| image| Hair Accessories| /data/beauty|Natural Beauty Ti...|Over The Years, P...|051e231c-7509-437...|
 education|Tired of Google C...| 1| 9000|https://i.pinimg....| 3813| image|Science of Curiosity| /data/education|Google Classroom,...|Fun Interactive D...|49f7dfe7-2e5a-403...|
 quotes|summcoco gives yo...| 1| 306000|https://i.pinimg....| 7922| image|Sumcoco | Decor I...| /data/quotes|Life Quotes Love,...|45 Top Life Quote...|a584581c-1b38-473...|
 christmas|Connecticut life ...| 1| 84000|https://i.pinimg....| 1643| image| Lauren McBride| /data/christmas|Christmas Dining ...|White and Gold Ch...|41be1bcd-1ead-476...|
 finance|Buying your first...| 1| 223|https://i.pinimg....| 5091| image|Edwin | Cash The ...| /data/finance|Paying Off Car Lo...|Buying Your First...|43459535-64e9-48a...|
 travel|Are you traveling...| 1| 3000|https://i.pinimg....| 9979| image| Petite in Paris| /data/travel|Torre Eiffel Pari...|Paris in the Summ...|2b2abc85-fc51-481...|
diy-and-crafts|This ghost craft ...| 1| 56000|https://i.pinimg....| 3032| video| Thriving Home|/data/diy-and-crafts|Halloween Arts An...| Puffy Ghost Craft|208b07bc-e042-4e8...|
 mens-fashion|It's safe to assu...| 1| 383000|https://i.pinimg....| 6809| image| Suburban Men| /data/mens-fashion|Business Casual D...|It's Time to Ditc...|f8ca698f-abfc-4b6...|
 education|Amazon is a teach...| 1| 41000|https://i.pinimg....| 3644| image|Teach Create Moti...| /data/education|Classroom Hacks,5...|More Amazon Must ...|cde2b4e2-7711-4a8...|
 home-decor|Believe it or not...| 1| 31000|https://i.pinimg....| 6575| image| Maria Louise Design| /data/home-decor|Farmhouse Remodel...|Affordable Farmho...|15c45907-657e-439...|
 home-decor|Holiday mantle de...| 1| 83000|https://i.pinimg....| 6145| image| Stylin by Aylin| /data/home-decor|Winter Home Decor...|HOLIDAY MANTLE DE...|82e13a07-db99-43a...|
 home-decor|Holiday mantle de...| 1| 83000|https://i.pinimg....| 6145| image| Stylin by Aylin| /data/home-decor|Winter Home Decor...|HOLIDAY MANTLE DE...|82e13a07-db99-43a...|
 art|Mexican artist En...| 1| null|https://i.pinimg....| 4| image| Bored Panda| /data/art|Girl Drawing Sket...|Mexican Artist Us...|55abcd28-bda1-445...|
 education|Podcasts for Teac...| 1| 25000|https://i.pinimg....| 4315| image| Math Giraffe| /data/education|Middle School Cla...|Podcasts for Teac...|21b59ba9-829d-4c3...|
event-planning|Sacramento Califo...| 1| null|https://i.pinimg....| 5069| image| Style Me Pretty|/data/event-planning|60th Anniversary ...|T

In [ ]:
# Reorder the DataFrame columns
df_pin = df_pin.select("ind", "unique_id", "title", "description", "follower_count", 
                       "poster_name", "tag_list", "is_image_or_video", "image_src", 
                       "save_location", "category")

# Show the DataFrame after reordering
df_pin.show()
display(df_pin.head())

+-----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list|is_image_or_video| image_src| save_location| category|
+-----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
10138|927c4658-cc3f-4b9...|14 Amazing Things...|This Costa Rica i...| 10000|Wanderlust Chloe ...|Costa Rica Travel...| image|https://i.pinimg....| /data/travel| travel|
 3156|fa6e31a4-18c2-4ec...|Handprint Reindee...|This post may con...| 892000|Michelle {CraftyM...|Christmas Gifts F...| image|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
 5395|71f72304-3708-45f...|So startest du mi...|Der #Vermögensauf...| 3000|DIVDepot – Geld s...|Money Saving Chal...| image|https://i.pinimg....| /data/finance| finance|
 5494|8fb2af68-543b-463...|Dave Ramsey's 7 B...|If you love budge...| 26000|Living Low Key | ...|Financial Peace,F...| image|https://i.pinimg....| /data/finance| finance|
 2418|da8745a6-5160-46c...|100 DIY Christmas...|Here are the best...| 500000| HikenDip|Farmhouse Christm...| image|https://i.pinimg....| /data/christmas| christmas|
 1611|051e231c-7509-437...|Over The Years, P...|[PaidLink] It's m...| 22| Hair Accessories|Natural Beauty Ti...| image|https://i.pinimg....| /data/beauty| beauty|
 3813|49f7dfe7-2e5a-403...|Fun Interactive D...|Tired of Google C...| 9000|Science of Curiosity|Google Classroom,...| image|https://i.pinimg....| /data/education| education|
 7922|a584581c-1b38-473...|45 Top Life Quote...|summcoco gives yo...| 306000|Sumcoco | Decor I...|Life Quotes Love,...| image|https://i.pinimg....| /data/quotes| quotes|
 1643|41be1bcd-1ead-476...|White and Gold Ch...|Connecticut life ...| 84000| Lauren McBride|Christmas Dining ...| image|https://i.pinimg....| /data/christmas| christmas|
 5091|43459535-64e9-48a...|Buying Your First...|Buying your first...| 223|Edwin | Cash The ...|Paying Off Car Lo...| image|https://i.pinimg....| /data/finance| finance|
 9979|2b2abc85-fc51-481...|Paris in the Summ...|Are you traveling...| 3000| Petite in Paris|Torre Eiffel Pari...| image|https://i.pinimg....| /data/travel| travel|
 3032|208b07bc-e042-4e8...| Puffy Ghost Craft|This ghost craft ...| 56000| Thriving Home|Halloween Arts An...| video|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
 6809|f8ca698f-abfc-4b6...|It's Time to Ditc...|It's safe to assu...| 383000| Suburban Men|Business Casual D...| image|https://i.pinimg....| /data/mens-fashion| mens-fashion|
 3644|cde2b4e2-7711-4a8...|More Amazon Must ...|Amazon is a teach...| 41000|Teach Create Moti...|Classroom Hacks,5...| image|https://i.pinimg....| /data/education| education|
 6575|15c45907-657e-439...|Affordable Farmho...|Believe it or not...| 31000| Maria Louise Design|Farmhouse Remodel...| image|https://i.pinimg....| /data/home-decor| home-decor|
 6145|82e13a07-db99-43a...|HOLIDAY MANTLE DE...|Holiday mantle de...| 83000| Stylin by Aylin|Winter Home Decor...| image|https://i.pinimg....| /data/home-decor| home-decor|
 6145|82e13a07-db99-43a...|HOLIDAY MANTLE DE...|Holiday mantle de...| 83000| Stylin by Aylin|Winter Home Decor...| image|https://i.pinimg....| /data/home-decor| home-decor|
 4|55abcd28-bda1-445...|Mexican Artist Us...|Mexican artist En...| null| Bored Panda|Girl Drawing Sket...| image|https://i.pinimg....| /data/art| art|
 4315|21b59ba9-829d-4c3...|Podcasts for Teac...|Podcasts for Teac...| 25000| Math Giraffe|Middle School Cla...| image|https://i.pinimg....| /data/education| education|
 5069|b75b6f87-deb3-444...|The Vault: Curate...|Sacramento Califo...| null| Style Me Pretty|60th Anniversary ...| image|https://i.pinimg....|/data/event-planning|event-planning|
+-----+--------------------+--------------------+--

## Task 2:

#### To clean the df_geo DataFrame you should perform the following transformations:

- Create a new column coordinates that contains an array based on the latitude and longitude columns
- Drop the latitude and longitude columns from the DataFrame
- Convert the timestamp column from a string to a timestamp data type
- Reorder the DataFrame columns to have the following column order:
  - ind
  - country
  - coordinates
  - timestamp

In [ ]:
# MuiTypography-root MuiTypography-body2 css-15fqxqr
from pyspark.sql.functions import array, col

# Create a new column coordinates containing an array of latitude and longitude
df_geo = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))

# Show the DataFrame after creating the coordinates column
df_geo.show()


+--------------------+-----+--------+---------+-------------------+--------------------+
 country| ind|latitude|longitude| timestamp| coordinates|
+--------------------+-----+--------+---------+-------------------+--------------------+
Antarctica (the t...| 2418|-88.4642| -171.061|2022-05-27 11:30:59|[-88.4642, -171.061]|
Bouvet Island (Bo...| 9270|-84.3984| -144.933|2022-05-23 03:24:24|[-84.3984, -144.933]|
Cocos (Keeling) I...|10794|-89.5236| -154.567|2022-01-01 02:26:50|[-89.5236, -154.567]|
British Virgin Is...| 5395|-6.93422| -24.9989|2020-08-15 19:30:19|[-6.93422, -24.9989]|
Central African R...| 2074|-52.3213| -50.11|2019-11-03 05:41:59| [-52.3213, -50.11]|
Libyan Arab Jamah...| 1170| 56.1198| 20.2963|2019-08-07 22:17:07| [56.1198, 20.2963]|
British Virgin Is...| 9550| -84.918| 7.23235|2022-03-23 00:25:45| [-84.918, 7.23235]|
 Antigua and Barbuda| 7922|-88.0974| -172.052|2021-01-27 09:14:19|[-88.0974, -172.052]|
 Dominican Republic| 9979| 14.9967| -120.682|2018-07-18 19:01:46| [14.9967, -120.682]|
 Dominican Republic| 6575| 81.7192| 61.5152|2020-12-29 22:59:37| [81.7192, 61.5152]|
 American Samoa| 3032|-66.7253| -122.489|2018-04-14 09:28:18|[-66.7253, -122.489]|
 American Samoa| 637|-41.2244| -158.022|2018-06-06 01:25:58|[-41.2244, -158.022]|
 Burkina Faso|10321|-89.1005| -151.255|2022-10-09 08:06:50|[-89.1005, -151.255]|
 Cote d'Ivoire| 2923|-84.6302| -164.507|2019-09-08 22:53:09|[-84.6302, -164.507]|
 French Guiana| 8304|-28.8852| -164.87|2019-09-13 04:50:29| [-28.8852, -164.87]|
 Cote d'Ivoire| 4315|-45.8508| 66.1003|2019-12-15 03:51:28| [-45.8508, 66.1003]|
 Afghanistan| 4249|-88.5478| -174.971|2021-09-01 11:10:02|[-88.5478, -174.971]|
 Bangladesh| 6809|-89.4254| -161.818|2020-04-17 13:20:43|[-89.4254, -161.818]|
 Azerbaijan| 5069|-63.0063| -157.474|2021-03-20 09:32:44|[-63.0063, -157.474]|
 Mozambique| 6145|-65.9079| -143.845|2019-12-05 02:09:44|[-65.9079, -143.845]|
+--------------------+-----+--------+---------+-------------------+--------------------+
only showing top 20 rows

In [ ]:
# Drop the latitude and longitude columns
df_geo = df_geo.drop("latitude", "longitude")

# Show the DataFrame after dropping the columns
df_geo.show()


+--------------------+-----+-------------------+--------------------+
 country| ind| timestamp| coordinates|
+--------------------+-----+-------------------+--------------------+
Antarctica (the t...| 2418|2022-05-27 11:30:59|[-88.4642, -171.061]|
Bouvet Island (Bo...| 9270|2022-05-23 03:24:24|[-84.3984, -144.933]|
Cocos (Keeling) I...|10794|2022-01-01 02:26:50|[-89.5236, -154.567]|
British Virgin Is...| 5395|2020-08-15 19:30:19|[-6.93422, -24.9989]|
Central African R...| 2074|2019-11-03 05:41:59| [-52.3213, -50.11]|
Libyan Arab Jamah...| 1170|2019-08-07 22:17:07| [56.1198, 20.2963]|
British Virgin Is...| 9550|2022-03-23 00:25:45| [-84.918, 7.23235]|
 Antigua and Barbuda| 7922|2021-01-27 09:14:19|[-88.0974, -172.052]|
 Dominican Republic| 9979|2018-07-18 19:01:46| [14.9967, -120.682]|
 Dominican Republic| 6575|2020-12-29 22:59:37| [81.7192, 61.5152]|
 American Samoa| 3032|2018-04-14 09:28:18|[-66.7253, -122.489]|
 American Samoa| 637|2018-06-06 01:25:58|[-41.2244, -158.022]|
 Burkina Faso|10321|2022-10-09 08:06:50|[-89.1005, -151.255]|
 Cote d'Ivoire| 2923|2019-09-08 22:53:09|[-84.6302, -164.507]|
 French Guiana| 8304|2019-09-13 04:50:29| [-28.8852, -164.87]|
 Cote d'Ivoire| 4315|2019-12-15 03:51:28| [-45.8508, 66.1003]|
 Afghanistan| 4249|2021-09-01 11:10:02|[-88.5478, -174.971]|
 Bangladesh| 6809|2020-04-17 13:20:43|[-89.4254, -161.818]|
 Azerbaijan| 5069|2021-03-20 09:32:44|[-63.0063, -157.474]|
 Mozambique| 6145|2019-12-05 02:09:44|[-65.9079, -143.845]|
+--------------------+-----+-------------------+--------------------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import to_timestamp

# Convert the timestamp column to a timestamp data type
df_geo = df_geo.withColumn("timestamp", to_timestamp("timestamp"))

# Show the DataFrame after converting the timestamp column
df_geo.show()


+--------------------+-----+-------------------+--------------------+
 country| ind| timestamp| coordinates|
+--------------------+-----+-------------------+--------------------+
Antarctica (the t...| 2418|2022-05-27 11:30:59|[-88.4642, -171.061]|
Bouvet Island (Bo...| 9270|2022-05-23 03:24:24|[-84.3984, -144.933]|
Cocos (Keeling) I...|10794|2022-01-01 02:26:50|[-89.5236, -154.567]|
British Virgin Is...| 5395|2020-08-15 19:30:19|[-6.93422, -24.9989]|
Central African R...| 2074|2019-11-03 05:41:59| [-52.3213, -50.11]|
Libyan Arab Jamah...| 1170|2019-08-07 22:17:07| [56.1198, 20.2963]|
British Virgin Is...| 9550|2022-03-23 00:25:45| [-84.918, 7.23235]|
 Antigua and Barbuda| 7922|2021-01-27 09:14:19|[-88.0974, -172.052]|
 Dominican Republic| 9979|2018-07-18 19:01:46| [14.9967, -120.682]|
 Dominican Republic| 6575|2020-12-29 22:59:37| [81.7192, 61.5152]|
 American Samoa| 3032|2018-04-14 09:28:18|[-66.7253, -122.489]|
 American Samoa| 637|2018-06-06 01:25:58|[-41.2244, -158.022]|
 Burkina Faso|10321|2022-10-09 08:06:50|[-89.1005, -151.255]|
 Cote d'Ivoire| 2923|2019-09-08 22:53:09|[-84.6302, -164.507]|
 French Guiana| 8304|2019-09-13 04:50:29| [-28.8852, -164.87]|
 Cote d'Ivoire| 4315|2019-12-15 03:51:28| [-45.8508, 66.1003]|
 Afghanistan| 4249|2021-09-01 11:10:02|[-88.5478, -174.971]|
 Bangladesh| 6809|2020-04-17 13:20:43|[-89.4254, -161.818]|
 Azerbaijan| 5069|2021-03-20 09:32:44|[-63.0063, -157.474]|
 Mozambique| 6145|2019-12-05 02:09:44|[-65.9079, -143.845]|
+--------------------+-----+-------------------+--------------------+
only showing top 20 rows

In [ ]:
# Reorder the DataFrame columns
df_geo = df_geo.select("ind", "country", "coordinates", "timestamp")

# Show the DataFrame after reordering the columns
df_geo.show()


+-----+--------------------+--------------------+-------------------+
 ind| country| coordinates| timestamp|
+-----+--------------------+--------------------+-------------------+
 2418|Antarctica (the t...|[-88.4642, -171.061]|2022-05-27 11:30:59|
 9270|Bouvet Island (Bo...|[-84.3984, -144.933]|2022-05-23 03:24:24|
10794|Cocos (Keeling) I...|[-89.5236, -154.567]|2022-01-01 02:26:50|
 5395|British Virgin Is...|[-6.93422, -24.9989]|2020-08-15 19:30:19|
 2074|Central African R...| [-52.3213, -50.11]|2019-11-03 05:41:59|
 1170|Libyan Arab Jamah...| [56.1198, 20.2963]|2019-08-07 22:17:07|
 9550|British Virgin Is...| [-84.918, 7.23235]|2022-03-23 00:25:45|
 7922| Antigua and Barbuda|[-88.0974, -172.052]|2021-01-27 09:14:19|
 9979| Dominican Republic| [14.9967, -120.682]|2018-07-18 19:01:46|
 6575| Dominican Republic| [81.7192, 61.5152]|2020-12-29 22:59:37|
 3032| American Samoa|[-66.7253, -122.489]|2018-04-14 09:28:18|
 637| American Samoa|[-41.2244, -158.022]|2018-06-06 01:25:58|
10321| Burkina Faso|[-89.1005, -151.255]|2022-10-09 08:06:50|
 2923| Cote d'Ivoire|[-84.6302, -164.507]|2019-09-08 22:53:09|
 8304| French Guiana| [-28.8852, -164.87]|2019-09-13 04:50:29|
 4315| Cote d'Ivoire| [-45.8508, 66.1003]|2019-12-15 03:51:28|
 4249| Afghanistan|[-88.5478, -174.971]|2021-09-01 11:10:02|
 6809| Bangladesh|[-89.4254, -161.818]|2020-04-17 13:20:43|
 5069| Azerbaijan|[-63.0063, -157.474]|2021-03-20 09:32:44|
 6145| Mozambique|[-65.9079, -143.845]|2019-12-05 02:09:44|
+-----+--------------------+--------------------+-------------------+
only showing top 20 rows

## Task 3:

In [ ]:
from pyspark.sql.functions import concat, col

# Create a new column user_name by concatenating first_name and last_name
df_user = df_user.withColumn("user_name", concat(col("first_name"), col("last_name")))

# Show the DataFrame with the new user_name column
df_user.show()


+---+-------------------+----------+-----+---------+------------------+
age| date_joined|first_name| ind|last_name| user_name|
+---+-------------------+----------+-----+---------+------------------+
 20|2015-10-23 04:13:23|Alexandria| 4249| Alvarado|AlexandriaAlvarado|
 21|2015-11-10 09:27:42| Andrea| 8731|Alexander| AndreaAlexander|
 21|2015-11-10 09:27:42| Andrea| 8731|Alexander| AndreaAlexander|
 21|2015-11-10 09:27:42| Andrea| 8731|Alexander| AndreaAlexander|
 34|2016-01-23 03:59:37| Daniel|10137| Matthews| DanielMatthews|
 21|2015-11-10 09:27:42| Andrea| 8731|Alexander| AndreaAlexander|
 24|2016-03-31 20:56:39| Austin| 8887|Rodriguez| AustinRodriguez|
 36|2015-12-20 16:38:13| Michelle| 4315| Prince| MichellePrince|
 32|2017-10-10 20:09:33| Christian|10625| Lang| ChristianLang|
 36|2015-12-20 16:38:13| Michelle| 4315| Prince| MichellePrince|
 40|2017-06-28 16:17:39| Michael| 1514| Perkins| MichaelPerkins|
 36|2015-12-20 16:38:13| Michelle| 4315| Prince| MichellePrince|
 32|2017-10-10 20:09:33| Christian|10625| Lang| ChristianLang|
 32|2017-10-10 20:09:33| Christian|10625| Lang| ChristianLang|
 22|2016-02-11 20:46:04| Jennifer| 9672| Hudson| JenniferHudson|
 32|2016-04-02 03:51:23| Brittany| 1313| Jones| BrittanyJones|
 32|2016-04-02 03:51:23| Brittany| 1313| Jones| BrittanyJones|
 34|2016-12-22 00:02:02| Thomas|10794| Turner| ThomasTurner|
 32|2016-04-02 03:51:23| Brittany| 1313| Jones| BrittanyJones|
 34|2016-12-22 00:02:02| Thomas|10794| Turner| ThomasTurner|
+---+-------------------+----------+-----+---------+------------------+
only showing top 20 rows

In [ ]:
# Drop the first_name and last_name columns
df_user = df_user.drop("first_name", "last_name")

# Show the DataFrame after dropping columns
df_user.show()


+---+-------------------+-----+------------------+
age| date_joined| ind| user_name|
+---+-------------------+-----+------------------+
 20|2015-10-23 04:13:23| 4249|AlexandriaAlvarado|
 21|2015-11-10 09:27:42| 8731| AndreaAlexander|
 21|2015-11-10 09:27:42| 8731| AndreaAlexander|
 21|2015-11-10 09:27:42| 8731| AndreaAlexander|
 34|2016-01-23 03:59:37|10137| DanielMatthews|
 21|2015-11-10 09:27:42| 8731| AndreaAlexander|
 24|2016-03-31 20:56:39| 8887| AustinRodriguez|
 36|2015-12-20 16:38:13| 4315| MichellePrince|
 32|2017-10-10 20:09:33|10625| ChristianLang|
 36|2015-12-20 16:38:13| 4315| MichellePrince|
 40|2017-06-28 16:17:39| 1514| MichaelPerkins|
 36|2015-12-20 16:38:13| 4315| MichellePrince|
 32|2017-10-10 20:09:33|10625| ChristianLang|
 32|2017-10-10 20:09:33|10625| ChristianLang|
 22|2016-02-11 20:46:04| 9672| JenniferHudson|
 32|2016-04-02 03:51:23| 1313| BrittanyJones|
 32|2016-04-02 03:51:23| 1313| BrittanyJones|
 34|2016-12-22 00:02:02|10794| ThomasTurner|
 32|2016-04-02 03:51:23| 1313| BrittanyJones|
 34|2016-12-22 00:02:02|10794| ThomasTurner|
+---+-------------------+-----+------------------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import to_timestamp

# Convert the date_joined column to a timestamp data type
df_user = df_user.withColumn("date_joined", to_timestamp("date_joined"))

# Show the DataFrame after converting the date_joined column
df_user.show()


+---+-------------------+-----+------------------+
age| date_joined| ind| user_name|
+---+-------------------+-----+------------------+
 20|2015-10-23 04:13:23| 4249|AlexandriaAlvarado|
 21|2015-11-10 09:27:42| 8731| AndreaAlexander|
 21|2015-11-10 09:27:42| 8731| AndreaAlexander|
 21|2015-11-10 09:27:42| 8731| AndreaAlexander|
 34|2016-01-23 03:59:37|10137| DanielMatthews|
 21|2015-11-10 09:27:42| 8731| AndreaAlexander|
 24|2016-03-31 20:56:39| 8887| AustinRodriguez|
 36|2015-12-20 16:38:13| 4315| MichellePrince|
 32|2017-10-10 20:09:33|10625| ChristianLang|
 36|2015-12-20 16:38:13| 4315| MichellePrince|
 40|2017-06-28 16:17:39| 1514| MichaelPerkins|
 36|2015-12-20 16:38:13| 4315| MichellePrince|
 32|2017-10-10 20:09:33|10625| ChristianLang|
 32|2017-10-10 20:09:33|10625| ChristianLang|
 22|2016-02-11 20:46:04| 9672| JenniferHudson|
 32|2016-04-02 03:51:23| 1313| BrittanyJones|
 32|2016-04-02 03:51:23| 1313| BrittanyJones|
 34|2016-12-22 00:02:02|10794| ThomasTurner|
 32|2016-04-02 03:51:23| 1313| BrittanyJones|
 34|2016-12-22 00:02:02|10794| ThomasTurner|
+---+-------------------+-----+------------------+
only showing top 20 rows

In [ ]:
# Reorder the DataFrame columns
df_user = df_user.select("ind", "user_name", "age", "date_joined")

# Show the DataFrame after reordering the columns
df_user.show()


+-----+------------------+---+-------------------+
 ind| user_name|age| date_joined|
+-----+------------------+---+-------------------+
 4249|AlexandriaAlvarado| 20|2015-10-23 04:13:23|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42|
10137| DanielMatthews| 34|2016-01-23 03:59:37|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42|
 8887| AustinRodriguez| 24|2016-03-31 20:56:39|
 4315| MichellePrince| 36|2015-12-20 16:38:13|
10625| ChristianLang| 32|2017-10-10 20:09:33|
 4315| MichellePrince| 36|2015-12-20 16:38:13|
 1514| MichaelPerkins| 40|2017-06-28 16:17:39|
 4315| MichellePrince| 36|2015-12-20 16:38:13|
10625| ChristianLang| 32|2017-10-10 20:09:33|
10625| ChristianLang| 32|2017-10-10 20:09:33|
 9672| JenniferHudson| 22|2016-02-11 20:46:04|
 1313| BrittanyJones| 32|2016-04-02 03:51:23|
 1313| BrittanyJones| 32|2016-04-02 03:51:23|
10794| ThomasTurner| 34|2016-12-22 00:02:02|
 1313| BrittanyJones| 32|2016-04-02 03:51:23|
10794| ThomasTurner| 34|2016-12-22 00:02:02|
+-----+------------------+---+-------------------+
only showing top 20 rows

## Task 4: Find the most popular category by country

#### Find the most popular Pinterest category people post to based on their country.
Your query should return a DataFrame that contains the following columns:
- country
- category
- category_count, a new column containing the desired query output

### Start

In [ ]:
from pyspark.sql.functions import col, max
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)

# Group by country and category, then count the occurrences of each category for each country
df_category_count = df_combined.groupBy(df_geo_alias["geo_country"], df_pin_alias["category"]).count()

# Create a window partitioned by country, ordered by the count descending
windowSpec = Window.partitionBy("geo_country").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each country partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each country
df_top_category_per_country = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Select the desired columns
# This is the erroneours line.
####### df_most_popular = df_most_popular.select("geo_country", "category", "count")

# Filter to get only the top category (rank = 1) for each country
df_most_popular = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_most_popular = df_most_popular.withColumnRenamed("geo_country", "country")
df_most_popular = df_most_popular.withColumnRenamed("count", "category_count")

# Display the result
df_most_popular.show()

+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Afghanistan| education| 1|
 Albania| mens-fashion| 4|
 Algeria| quotes| 1|
 American Samoa|diy-and-crafts| 1|
 Angola|diy-and-crafts| 4|
 Anguilla| home-decor| 1|
Antarctica (the t...| christmas| 1|
 Antigua and Barbuda| quotes| 1|
 Armenia|diy-and-crafts| 5|
 Aruba| travel| 1|
 Australia| mens-fashion| 1|
 Austria|event-planning| 1|
 Azerbaijan|event-planning| 1|
 Bahamas| art| 1|
 Bangladesh| mens-fashion| 1|
 Barbados| travel| 4|
 Botswana| tattoos| 1|
Bouvet Island (Bo...| tattoos| 1|
British Virgin Is...| travel| 1|
 Bulgaria| finance| 1|
+--------------------+--------------+--------------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import col, max
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)

# Group by country and category, then count the occurrences of each category for each country
df_category_count = df_combined.groupBy(df_geo_alias["geo_country"], df_pin_alias["category"]).count()

# Create a window partitioned by country, ordered by the count descending
windowSpec = Window.partitionBy("geo_country").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each country partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each country
df_top_category_per_country = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Select the desired columns
# This is the erroneours line.
####### df_most_popular = df_most_popular.select("geo_country", "category", "count")

# Filter to get only the top category (rank = 1) for each country
df_most_popular = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_most_popular = df_most_popular.withColumnRenamed("geo_country", "country")
df_most_popular = df_most_popular.withColumnRenamed("count", "category_count")

# Display the result
df_most_popular.show()

+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Afghanistan| education| 1|
 Albania| mens-fashion| 4|
 Algeria| quotes| 1|
 American Samoa| art| 1|
 Angola|diy-and-crafts| 4|
 Anguilla| home-decor| 1|
Antarctica (the t...| christmas| 1|
 Antigua and Barbuda| quotes| 1|
 Armenia|diy-and-crafts| 5|
 Aruba| travel| 1|
 Australia| mens-fashion| 1|
 Austria| travel| 1|
 Azerbaijan|event-planning| 1|
 Bahamas| art| 1|
 Bangladesh| art| 1|
 Barbados| travel| 4|
 Botswana| tattoos| 1|
Bouvet Island (Bo...| tattoos| 1|
British Virgin Is...| travel| 1|
 Bulgaria| finance| 1|
+--------------------+--------------+--------------+
only showing top 20 rows

### End

In [ ]:
from pyspark.sql.functions import col

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)
print(df_combined)
# Group by country and category, then count the occurrences of each category for each country
df_category_count = df_combined.groupBy("geo_country", "pin.category").count()
print(df_category_count)
# Find the maximum count for each country
max_counts = df_category_count.groupBy("geo_country").max("count")
print(max_counts)

max_counts.show()


DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string, ind: bigint, geo_country: string, coordinates: array<double>, timestamp: timestamp]
DataFrame[geo_country: string, category: string, count: bigint]
DataFrame[geo_country: string, max(count): bigint]
+--------------------+----------+
 geo_country|max(count)|
+--------------------+----------+
 Anguilla| 1|
 Fiji| 1|
 Cambodia| 1|
 Afghanistan| 1|
 Maldives| 1|
 Sudan| 1|
British Virgin Is...| 1|
 Algeria| 1|
 Angola| 4|
 Albania| 4|
 India| 1|
 Bahamas| 1|
 American Samoa| 1|
 Burundi| 1|
Central African R...| 1|
 Bangladesh| 1|
 Barbados| 4|
 Congo| 1|
Cocos (Keeling) I...| 1|
 Cote d'Ivoire| 1|
+--------------------+----------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import col

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)
print(df_combined)
# Group by country and category, then count the occurrences of each category for each country
df_category_count = df_combined.groupBy("geo_country", "pin.category").count()
print(df_category_count)
# Find the maximum count for each country
max_counts = df_category_count.groupBy("geo_country").max("count")
print(max_counts)

max_counts.show()

DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string, ind: bigint, geo_country: string, coordinates: array<double>, timestamp: timestamp]
DataFrame[geo_country: string, category: string, count: bigint]
DataFrame[geo_country: string, max(count): bigint]
+--------------------+----------+
 geo_country|max(count)|
+--------------------+----------+
 Anguilla| 1|
 Fiji| 1|
 Afghanistan| 1|
 Cambodia| 1|
 Maldives| 1|
 Sudan| 1|
British Virgin Is...| 1|
 Algeria| 1|
 Angola| 4|
 Albania| 4|
 India| 1|
 Bahamas| 1|
 American Samoa| 1|
 Burundi| 1|
Central African R...| 1|
 Bangladesh| 1|
 Barbados| 4|
 Congo| 1|
Cocos (Keeling) I...| 1|
 Cote d'Ivoire| 1|
+--------------------+----------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import col

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")
print(df_pin_alias)
print(df_geo_alias)

DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string]
DataFrame[ind: bigint, geo_country: string, coordinates: array<double>, timestamp: timestamp]

## Task 5: Find which was the most popular category each year

Task 5: Find which was the most popular category each year

In [ ]:
from pyspark.sql.functions import col, year
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Combine the data from df_pin and df_geo
df_combined = df_pin.join(df_geo, df_pin.ind == df_geo.ind)

# Extract the year from the timestamp column
df_combined = df_combined.withColumn("year", year("timestamp"))

# Group by year and category, then count the occurrences of each category for each year
df_category_count = df_combined.groupBy("year", "category").count()

# Create a window partitioned by year, ordered by the count descending
windowSpec = Window.partitionBy("year").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each year partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each year
df_top_category_per_year = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
#df_top_category_per_year = df_top_category_per_year.withColumnRenamed("category", "most_popular_category")
df_top_category_per_year = df_top_category_per_year.withColumnRenamed("count", "category_count")

# Display the result
df_top_category_per_year.show()


+----+--------------+--------------+
year| category|category_count|
+----+--------------+--------------+
2018| vehicles| 4|
2019|diy-and-crafts| 5|
2020| mens-fashion| 6|
2021| quotes| 2|
2022| christmas| 2|
+----+--------------+--------------+

In [ ]:
from pyspark.sql.functions import col, year
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Combine the data from df_pin and df_geo
df_combined = df_pin.join(df_geo, df_pin.ind == df_geo.ind)

# Extract the year from the timestamp column and rename it to post_year
df_combined = df_combined.withColumn("post_year", year("timestamp")).drop("year")

# Group by post_year and category, then count the occurrences of each category for each post_year
df_category_count = df_combined.groupBy("post_year", "category").count()

# Create a window partitioned by post_year, ordered by the count descending
windowSpec = Window.partitionBy("post_year").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each post_year partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each post_year
df_top_category_per_year = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_top_category_per_year = df_top_category_per_year.withColumnRenamed("count", "category_count")

# Display the result
df_top_category_per_year.show()


+---------+--------------+--------------+
post_year| category|category_count|
+---------+--------------+--------------+
 2018| vehicles| 4|
 2019|diy-and-crafts| 5|
 2020| mens-fashion| 6|
 2021| tattoos| 2|
 2022| christmas| 2|
+---------+--------------+--------------+

## Task 6: Find the user with most followers in each country

### Step 1: For each country find the user with the most followers.
Your query should return a DataFrame that contains the following columns:

- country
- poster_name
- follower_count

In [ ]:
from pyspark.sql.functions import col, max
from pyspark.sql.window import Window

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)

# Group by country and user (poster_name), then find the maximum follower count for each group
df_max_followers_per_user = df_combined.groupBy("geo_country", "poster_name").agg(max("follower_count").alias("follower_count"))

# Create a window partitioned by country, ordered by the follower_count descending
windowSpec = Window.partitionBy("geo_country").orderBy(col("follower_count").desc())

# Use row_number to assign a rank to each row within each country partition
df_ranked_users = df_max_followers_per_user.withColumn("rank", row_number().over(windowSpec))

# Filter to keep only the top user (rank = 1) for each country
df_top_users_per_country = df_ranked_users.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_top_users_per_country = df_top_users_per_country.withColumnRenamed("geo_country", "country")

# Select the desired columns
df_top_users_per_country = df_top_users_per_country.select("country", "poster_name", "follower_count")

# Display the result
df_top_users_per_country.show()



+--------------------+--------------------+--------------+
 country| poster_name|follower_count|
+--------------------+--------------------+--------------+
 Afghanistan| Walmart| null|
 Albania| Bored Panda| null|
 Algeria| YourTango| 942000|
 American Samoa| Thriving Home| 56000|
 Angola| CraftGossip.com| 502000|
 Anguilla|Kristen | Lifesty...| 92000|
Antarctica (the t...| HikenDip| 500000|
 Antigua and Barbuda|Sumcoco | Decor I...| 306000|
 Armenia|Michelle {CraftyM...| 892000|
 Aruba| TheTrendSpotter| 211000|
 Australia| Cultura Colectiva| null|
 Austria|Wanderlust Chloe ...| 10000|
 Azerbaijan| Style Me Pretty| null|
 Bahamas|The Kitchen Table...| 221000|
 Bangladesh| Suburban Men| 383000|
 Barbados| Nicki | 28000|
 Botswana| RapidLeaks| 4000|
Bouvet Island (Bo...| POPSUGAR| null|
British Virgin Is...|Visit USA Parks |...| 4000|
 Bulgaria|Living Low Key | ...| 26000|
+--------------------+--------------------+--------------+
only showing top 20 rows

## Step 2: Based on the above query, find the country with the user with most followers.
Your query should return a DataFrame that contains the following columns:
- country
- follower_count

This DataFrame should have only one entry.

In [ ]:
from pyspark.sql.functions import col, max

# Group by country and find the maximum follower count across all users in each country
df_max_followers_per_country = df_top_users_per_country.groupBy("country").agg(max("follower_count").alias("follower_count"))

# Create a window partitioned by country, ordered by the follower_count descending
windowSpec = Window.orderBy(col("follower_count").desc())

# Use row_number to assign a rank to each row
df_ranked_countries = df_max_followers_per_country.withColumn("rank", row_number().over(windowSpec))

# Filter to keep only the top country (rank = 1)
df_country_with_most_followers = df_ranked_countries.filter(col("rank") == 1).drop("rank")

# Display the result
df_country_with_most_followers.show()


+-------+--------------+
country|follower_count|
+-------+--------------+
Algeria| 942000|
+-------+--------------+

### Task 7: Find the most popular category for different age groups

What is the most popular category people post to based on the following age groups:
- 18-24
- 25-35
- 36-50
- +50

Your query should return a DataFrame that contains the following columns:
- age_group, a new column based on the original age column
- category
- category_count, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import when

# Define age groups based on the original age column
df_user_age_grouped = df_user.withColumn("age_group",
                                         when(col("age").between(18, 24), "18-24")
                                         .when(col("age").between(25, 35), "25-35")
                                         .when(col("age").between(36, 50), "36-50")
                                         .otherwise("50+"))

# Join the combined DataFrame with df_pin on the index column
df_combined = df_pin.join(df_user_age_grouped, df_pin.ind == df_user_age_grouped.ind)

# Group by age group and category, then count the occurrences of each category for each age group
df_category_count = df_combined.groupBy("age_group", "category").count()

# Create a window partitioned by age group, ordered by the count descending
windowSpec = Window.partitionBy("age_group").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each age group partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each age group
df_most_popular_per_age_group = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_most_popular_per_age_group = df_most_popular_per_age_group.withColumnRenamed("count", "category_count")

# Display the result
df_most_popular_per_age_group.show()


+---------+--------------+--------------+
age_group| category|category_count|
+---------+--------------+--------------+
 18-24| mens-fashion| 17|
 25-35|diy-and-crafts| 12|
 36-50| finance| 5|
 50+| beauty| 1|
+---------+--------------+--------------+

### Task 8: Find the median follower count for different age groups

What is the median follower count for users in the following age groups:

- 18-24
- 25-35
- 36-50
- +50

Your query should return a DataFrame that contains the following columns:

- age_group, a new column based on the original age column
- median_follower_count, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import col, expr

# Define age groups based on the original age column
df_user_age_grouped = df_user.withColumn("age_group",
                                         when(col("age").between(18, 24), "18-24")
                                         .when(col("age").between(25, 35), "25-35")
                                         .when(col("age").between(36, 50), "36-50")
                                         .otherwise("50+"))

# Join the user DataFrame with the pin DataFrame on the index column
df_combined = df_pin.join(df_user_age_grouped, df_pin.ind == df_user_age_grouped.ind)

# Group by age group and calculate the median follower count for each age group
df_median_follower_count = df_combined.groupBy("age_group") \
    .agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

# Display the result
df_median_follower_count.show()


+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 18-24| 211000|
 25-35| 43000|
 36-50| 6000|
 50+| 908|
+---------+---------------------+

## Task 9: Find how many users have joined each year?
Find how many users have joined between 2015 and 2020.

Your query should return a DataFrame that contains the following columns:

- post_year, a new column that contains only the year from the timestamp column
- number_users_joined, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import year, countDistinct

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Group by the join year and count the distinct users
df_number_users_joined = df_filtered_users.groupBy("join_year").agg(countDistinct("user_name").alias("number_users_joined"))

# Display the result
df_number_users_joined.show()


+---------+-------------------+
join_year|number_users_joined|
+---------+-------------------+
 2015| 24|
 2016| 27|
 2017| 8|
+---------+-------------------+

## Task 10: Find the median follower count of users based on their joining year

Find the median follower count of users have joined between 2015 and 2020.

Your query should return a DataFrame that contains the following columns:

- post_year, a new column that contains only the year from the timestamp column
- median_follower_count, a new column containing the desired query output


In [ ]:
from pyspark.sql.functions import year
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate the median follower count for each year
windowSpec = Window.partitionBy("join_year")
df_median_follower_count = df_filtered_users.withColumn("median_follower_count", F.expr("percentile_approx(age, 0.5)").over(windowSpec)).distinct()

# Display the result
df_median_follower_count.show()


+-----+------------------+---+-------------------+---------+---------------------+
 ind| user_name|age| date_joined|join_year|median_follower_count|
+-----+------------------+---+-------------------+---------+---------------------+
 5730| RachelDavis| 36|2015-12-08 20:02:43| 2015| 22|
 2923| BrianNelson| 26|2015-11-11 03:20:57| 2015| 22|
 857| AndrewBurke| 20|2015-11-14 17:38:31| 2015| 22|
 7528| AbigailAli| 20|2015-10-24 11:23:51| 2015| 22|
 4249|AlexandriaAlvarado| 20|2015-10-23 04:13:23| 2015| 22|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42| 2015| 22|
 6063| CoreyAndrews| 23|2015-11-25 13:36:22| 2015| 22|
 6145| EmilyHawkins| 30|2015-12-25 20:24:37| 2015| 22|
 8304| CharlesBerry| 25|2015-12-28 04:21:39| 2015| 22|
 3156| AndrewBaker| 22|2015-12-21 08:06:54| 2015| 22|
 428| ClaudiaAdams| 20|2015-11-28 02:20:29| 2015| 22|
10138| CarolSilva| 22|2015-12-31 14:57:02| 2015| 22|
 7922| DeniseAdams| 21|2015-11-12 06:21:36| 2015| 22|
 9875| BrendanJoseph| 26|2015-12-20 10:28:00| 2015| 22|
 4315| MichellePrince| 36|2015-12-20 16:38:13| 2015| 22|
 8237| AaronAbbott| 20|2015-10-23 16:08:41| 2015| 22|
 3089| AbigailAli| 20|2015-10-24 11:23:51| 2015| 22|
 5494| AnneAllen| 27|2015-12-16 15:20:05| 2015| 22|
10883| CarlHampton| 25|2015-11-26 03:43:03| 2015| 22|
10321| AndreaBurke| 20|2015-12-12 11:20:08| 2015| 22|
+-----+------------------+---+-------------------+---------+---------------------+
only showing top 20 rows

## Task 11: Find the median follower count of users based on their joining year and age group

Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.
Your query should return a DataFrame that contains the following columns:

- age_group, a new column based on the original age column
- post_year, a new column that contains only the year from the timestamp column
- median_follower_count, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import year, expr, col
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate age group
df_filtered_users = df_filtered_users.withColumn("age_group", F.when(col("age") < 20, "0-19")
                                                     .when((col("age") >= 20) & (col("age") < 30), "20-29")
                                                     .when((col("age") >= 30) & (col("age") < 40), "30-39")
                                                     .otherwise("40+"))

# Group by age group, join year, and calculate the median follower count
df_median_follower_count = df_filtered_users.groupBy("age_group", "join_year") \
    .agg(expr("percentile_approx(ind, 0.5)").alias("median_follower_count"))

# Add a column for post_year containing only the year from the timestamp column
df_median_follower_count = df_median_follower_count.withColumn("post_year", col("join_year"))

# Reorder columns
df_median_follower_count = df_median_follower_count.select("age_group", "post_year", "median_follower_count")

# Display the result
df_median_follower_count.show()


+---------+---------+---------------------+
age_group|post_year|median_follower_count|
+---------+---------+---------------------+
 20-29| 2015| 7528|
 40+| 2016| 3813|
 20-29| 2016| 5069|
 30-39| 2016| 2863|
 20-29| 2017| 5091|
 30-39| 2015| 5730|
 40+| 2017| 1611|
 30-39| 2017| 10625|
+---------+---------+---------------------+

In [ ]:
from pyspark.sql.functions import year, expr, col
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate age group
#df_filtered_users = df_filtered_users.withColumn("age_group", F.when(col("age") < 20, "0-19")
                                                     #.when((col("age") >= 20) & (col("age") < 30), #"20-29")
                                                     #.when((col("age") >= 30) & (col("age") < 40), #"30-39")
                                                     #.otherwise("40+"))

# Group by age group, join year, and calculate the median follower count
df_median_follower_count = df_filtered_users.groupBy("join_year") \
    .agg(expr("percentile_approx(ind, 0.5)").alias("median_follower_count"))

# Display the result
df_median_follower_count.show()

+---------+---------------------+
join_year|median_follower_count|
+---------+---------------------+
 2015| 6145|
 2016| 4949|
 2017| 5091|
+---------+---------------------+

In [ ]:
from pyspark.sql.functions import year, expr, col
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate age group
df_filtered_users = df_filtered_users.withColumn("age_group", F.when(col("age") < 20, "0-19")
                                                     .when((col("age") >= 20) & (col("age") < 30), "20-29")
                                                     .when((col("age") >= 30) & (col("age") < 40), "30-39")
                                                     .otherwise("40+"))

# Group by age group, join year, and calculate the median follower count
df_median_follower_count = df_filtered_users.groupBy("age_group", "join_year") \
    .agg(expr("percentile_approx(ind, 0.5)").alias("median_follower_count"))

# Display the result
df_median_follower_count.show()


+---------+---------+---------------------+
age_group|join_year|median_follower_count|
+---------+---------+---------------------+
 30-39| 2016| 2863|
 20-29| 2015| 7528|
 30-39| 2015| 5730|
 20-29| 2016| 5069|
 40+| 2017| 1611|
 20-29| 2017| 5091|
 40+| 2016| 3813|
 30-39| 2017| 10625|
+---------+---------+---------------------+

In [ ]:
from pyspark.sql.functions import expr
from pyspark.sql.window import Window

# Assuming follower information is contained within df_pin, you can filter the relevant columns
df_followers = df_pin.select("ind", "follower_count")

# Now you have a DataFrame df_followers containing the unique_id and follower_count columns


# Assuming df_followers is properly defined and contains the necessary columns
# Join df_filtered_users with df_followers on a common key (e.g., unique_id)
print(df_filtered_users)
print(df_followers)
print(df_followers.columns)  # This line is causing the error

# Correcting the error: Accessing the columns attribute of df_followers
print(df_followers.columns)


df_filtered_users_with_followers = df_filtered_users.join(df_followers, df_filtered_users["ind"] == df_followers["ind"], how="inner")
print(df_filtered_users)
print(df_followers)
# Once the join is successful, you can proceed with the rest of the code to calculate the median follower count

# Calculate the median follower count for each year and age group
windowSpec = Window.partitionBy("join_year", "age_group")
df_median_follower_count = df_filtered_users_with_followers.withColumn(
    "median_follower_count",
    expr("percentile_approx(follower_count, 0.5)").over(windowSpec)
).select("age_group", "join_year", "median_follower_count").distinct()

# Display the result
df_median_follower_count.show()


DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp, join_year: int, age_group: string]
DataFrame[ind: bigint, follower_count: int]
['ind', 'follower_count']
['ind', 'follower_count']
DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp, join_year: int, age_group: string]
DataFrame[ind: bigint, follower_count: int]
+---------+---------+---------------------+
age_group|join_year|median_follower_count|
+---------+---------+---------------------+
 18-24| 2015| 211000|
 25-35| 2015| 51000|
 36-50| 2015| 0|
 18-24| 2016| 502000|
 25-35| 2016| 43000|
 36-50| 2016| 17000|
 50+| 2016| 908|
 18-24| 2017| 940|
 25-35| 2017| 8000|
 36-50| 2017| 6000|
+---------+---------+---------------------+

In [ ]:
# Assuming df_followers is properly defined and contains the necessary columns
# Join df_filtered_users with df_followers on a common key (e.g., unique_id)
print(df_filtered_users)
print(df_followers)
print(df_followers.columns)  # This line is causing the error

# Correcting the error: Accessing the columns attribute of df_followers
print(df_followers.columns)

df_filtered_users_with_followers = df_filtered_users.join(df_followers, df_filtered_users["ind"] == df_followers["ind"], how="inner")
print(df_filtered_users)
print(df_followers)
# Once the join is successful, you can proceed with the rest of the code to calculate the median follower count

# Calculate the median follower count for each year and age group
windowSpec = Window.partitionBy("join_year", "age_group")
df_median_follower_count = df_filtered_users_with_followers.withColumn(
    "median_follower_count",
    expr("percentile_approx(follower_count, 0.5)").over(windowSpec)
).select("age_group", "join_year", "median_follower_count").distinct()

# Display the result
df_median_follower_count.show()


DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp, join_year: int, age_group: string]
DataFrame[ind: bigint, follower_count: int]
['ind', 'follower_count']
['ind', 'follower_count']
DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp, join_year: int, age_group: string]
DataFrame[ind: bigint, follower_count: int]
+---------+---------+---------------------+
age_group|join_year|median_follower_count|
+---------+---------+---------------------+
 18-24| 2015| 211000|
 25-35| 2015| 51000|
 36-50| 2015| 0|
 18-24| 2016| 502000|
 25-35| 2016| 43000|
 36-50| 2016| 17000|
 50+| 2016| 908|
 18-24| 2017| 940|
 25-35| 2017| 8000|
 36-50| 2017| 6000|
+---------+---------+---------------------+

In [ ]:
print(df_pin)

DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string]

In [ ]:
from pyspark.sql.functions import year
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate the median follower count for each year
windowSpec = Window.partitionBy("join_year")
df_median_follower_count = df_filtered_users.withColumn("median_follower_count", F.expr("percentile_approx(age, 0.5)").over(windowSpec)).distinct()

# Display the result
df_median_follower_count.show()

+-----+---------------+---+-------------------+---------+---------------------+
 ind| user_name|age| date_joined|join_year|median_follower_count|
+-----+---------------+---+-------------------+---------+---------------------+
 4315| MichellePrince| 36|2015-12-20 16:38:13| 2015| 22|
 3089| AbigailAli| 20|2015-10-24 11:23:51| 2015| 22|
 7528| AbigailAli| 20|2015-10-24 11:23:51| 2015| 22|
 5494| AnneAllen| 27|2015-12-16 15:20:05| 2015| 22|
 9875| BrendanJoseph| 26|2015-12-20 10:28:00| 2015| 22|
 8304| CharlesBerry| 25|2015-12-28 04:21:39| 2015| 22|
 9270| AmyBrown| 21|2015-11-08 16:14:16| 2015| 22|
 4| AdamAcosta| 20|2015-10-21 21:26:45| 2015| 22|
10029| AnthonyBaker| 20|2015-11-14 19:24:56| 2015| 22|
 1667| BernardArnold| 21|2015-12-12 01:39:29| 2015| 22|
 5730| RachelDavis| 36|2015-12-08 20:02:43| 2015| 22|
 2923| BrianNelson| 26|2015-11-11 03:20:57| 2015| 22|
 2418| AmandaAdams| 20|2015-10-21 08:27:36| 2015| 22|
 857| AndrewBurke| 20|2015-11-14 17:38:31| 2015| 22|
 6063| CoreyAndrews| 23|2015-11-25 13:36:22| 2015| 22|
 6145| EmilyHawkins| 30|2015-12-25 20:24:37| 2015| 22|
 8731|AndreaAlexander| 21|2015-11-10 09:27:42| 2015| 22|
 8237| AaronAbbott| 20|2015-10-23 16:08:41| 2015| 22|
 3156| AndrewBaker| 22|2015-12-21 08:06:54| 2015| 22|
 428| ClaudiaAdams| 20|2015-11-28 02:20:29| 2015| 22|
+-----+---------------+---+-------------------+---------+---------------------+
only showing top 20 rows